In [1]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix

C:\Users\ester\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [28]:
base_dir_train_crack = 'dataset/train/crack/'
base_dir_train_non_crack = 'dataset/train/non-crack-subset/'
base_dir_test_crack = 'dataset/test/crack/'
base_dir_test_non_crack = 'dataset/test/non-crack-subset/'
base_dir_processed_train_images = 'processed-dataset-train/'

In [29]:
preprocessed_image_name_prefix = 'preprocessed_'

Obtendo listas com o nome das imagens de treino e teste

In [15]:
crack_images_for_train_name_list = os.listdir(base_dir_train_crack)
non_crack_images_for_train_name_list = os.listdir(base_dir_train_non_crack)

In [16]:
crack_images_for_test_name_list = os.listdir(base_dir_test_crack)
non_crack_images_for_test_name_list = os.listdir(base_dir_test_non_crack)

In [17]:
def load_images_from_name_list(base_dir, list_with_names, default_image_width, default_image_height, cv2_imread_colorscheme):
    loaded_images = []
    for image_name in list_with_names:
        im = cv2.imread(base_dir + image_name, cv2_imread_colorscheme)
        im_resized = cv2.resize(im, (default_image_width, default_image_height))
        loaded_images.append(im_resized)
    return loaded_images

Definindo constantes (tamanho das imagens e esquema de cores)

In [18]:
default_image_width = 256
default_image_height = 256
default_colorscheme = cv2.IMREAD_GRAYSCALE

Carregando as imagens para o treinamento

In [19]:
crack_images_for_train_list = load_images_from_name_list(base_dir_train_crack, \
                                                         crack_images_for_train_name_list, \
                                                         default_image_width, default_image_height, \
                                                         default_colorscheme)

In [20]:
non_crack_images_for_train_list = load_images_from_name_list(base_dir_train_non_crack, \
                                                             non_crack_images_for_train_name_list, \
                                                             default_image_width, default_image_height, \
                                                             default_colorscheme)

In [25]:
median_blur_kernel_size = 3
clahe_clip_limit = 4.0
clahe_tile_grid_size = (30, 30)
first_erosion_iterations = 3
first_dilation_iterations = 2
size_for_erosion_and_dilation_element = 2

element_for_erosion_and_dilation = \
    cv2.getStructuringElement(cv2.MORPH_CROSS, \
                              (2 * size_for_erosion_and_dilation_element + 1, 2 * size_for_erosion_and_dilation_element + 1), \
                              (size_for_erosion_and_dilation_element, size_for_erosion_and_dilation_element))

addapt_thresh_neighbour_size = 11
addapt_thresh_subtraction_constant = 2


Função que aplica o pré processamento nas imagens

In [34]:
def pre_process_images_from_list(images_list, \
                                 median_blur_kernel_size, \
                                 clahe_clip_limit, \
                                 clahe_tile_grid_size, \
                                 first_erosion_iterations, \
                                 element_for_erosion_and_dilation, \
                                 first_dilation_iterations, \
                                 addapt_thresh_neighbour_size, addapt_thresh_subtraction_constant):
    
    pre_processed_images = []
    clahe = cv2.createCLAHE(clipLimit = clahe_clip_limit, tileGridSize = clahe_tile_grid_size)
    
    for image in images_list:
        blured_image = cv2.medianBlur(image, median_blur_kernel_size) # aplica filtro de mediana
        clahe_image = clahe.apply(blured_image) # aplica equalizacao de histograma por blocos, uniformiza iluminacao
        min_pixel_value = np.min(clahe_image) # obtem o pixel de menor valor na imagem
        darker_image = clahe_image - min_pixel_value # escurece toda a imagem, subtraindo o valor do menor pixel
        
        erosion_image = cv2.erode(darker_image, \
                                  element_for_erosion_and_dilation, \
                                  iterations = first_erosion_iterations) # aplica 2 erosoes
        
        dilated_image = cv2.dilate(erosion_image, \
                                   element_for_erosion_and_dilation, \
                                   iterations = first_dilation_iterations) # aplica 2 dilatacoes
        
        image_threshold = cv2.adaptiveThreshold(dilated_image, 255, \
                                                cv2.ADAPTIVE_THRESH_MEAN_C, \
                                                cv2.THRESH_BINARY, \
                                                addapt_thresh_neighbour_size, \
                                                addapt_thresh_subtraction_constant)
        
        #image_negative = cv2.bitwise_not(image_threshold)
        pre_processed_images.append(image_threshold)
        
    return pre_processed_images

Pré-processando as imagens para treinamento do modelo

In [35]:
pre_processed_train_crack = pre_process_images_from_list(crack_images_for_train_list, \
                                                         median_blur_kernel_size, \
                                                         clahe_clip_limit, \
                                                         clahe_tile_grid_size, \
                                                         first_erosion_iterations, \
                                                         element_for_erosion_and_dilation, \
                                                         first_dilation_iterations, \
                                                         addapt_thresh_neighbour_size, \
                                                         addapt_thresh_subtraction_constant)


In [36]:
pre_processed_train_non_crack = pre_process_images_from_list(non_crack_images_for_train_list, \
                                                         median_blur_kernel_size, \
                                                         clahe_clip_limit, \
                                                         clahe_tile_grid_size, \
                                                         first_erosion_iterations, \
                                                         element_for_erosion_and_dilation, \
                                                         first_dilation_iterations, \
                                                         addapt_thresh_neighbour_size, \
                                                         addapt_thresh_subtraction_constant)


Função para salvar imagens

In [30]:
def save_images(base_dir, images_list, images_names_list, image_name_prefix, directory_to_save):
    for idx in range(len(images_names_list)):
        
        if not os.path.exists(base_dir + directory_to_save):
            os.makedirs(base_dir + directory_to_save)
            
        cv2.imwrite(base_dir + directory_to_save + '/' + image_name_prefix + images_names_list[idx], images_list[idx])
        

Salvando as imagens de treinamento pré-processadas

In [38]:
save_images(base_dir_processed_train_images, \
            pre_processed_train_crack, \
            crack_images_for_train_name_list, \
            preprocessed_image_name_prefix, \
            'crack')


In [39]:
save_images(base_dir_processed_train_images, \
            pre_processed_train_non_crack, \
            non_crack_images_for_train_name_list, \
            preprocessed_image_name_prefix, \
            'non-crack')


Treinando modelo Support Vector Machine

In [40]:
svm_input_masks = []
svm_input_masks.extend(pre_processed_train_crack)
svm_input_masks.extend(pre_processed_train_non_crack) # adicionamos todas as imagens à lista de input do modelo
svm_input_labels = []
svm_input_labels.extend(np.ones((len(pre_processed_train_crack), ), np.uint8))
svm_input_labels.extend(np.zeros((len(pre_processed_train_non_crack), ), np.uint8)) # adicionamos as labels das imagens

svm_input_masks = list(map(lambda x:x.flatten(), svm_input_masks)) # aqui, redimensionamos as imagens para terem 1 dimensao

In [41]:
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(svm_input_masks, svm_input_labels)

SVC(kernel='linear')

In [43]:
crack_images_for_test_list = load_images_from_name_list(base_dir_test_crack, \
                                                        crack_images_for_test_name_list, \
                                                        default_image_width, default_image_height, \
                                                        default_colorscheme)

In [45]:
non_crack_images_for_test_list = load_images_from_name_list(base_dir_test_non_crack, \
                                                            non_crack_images_for_test_name_list, \
                                                            default_image_width, default_image_height, \
                                                            default_colorscheme)

In [44]:
pre_processed_test_crack = pre_process_images_from_list(crack_images_for_test_list, \
                                                        median_blur_kernel_size, \
                                                        clahe_clip_limit, \
                                                        clahe_tile_grid_size, \
                                                        first_erosion_iterations, \
                                                        element_for_erosion_and_dilation, \
                                                        first_dilation_iterations, \
                                                        addapt_thresh_neighbour_size, \
                                                        addapt_thresh_subtraction_constant)

In [46]:
pre_processed_test_non_crack = pre_process_images_from_list(non_crack_images_for_test_list, \
                                                            median_blur_kernel_size, \
                                                            clahe_clip_limit, \
                                                            clahe_tile_grid_size, \
                                                            first_erosion_iterations, \
                                                            element_for_erosion_and_dilation, \
                                                            first_dilation_iterations, \
                                                            addapt_thresh_neighbour_size, \
                                                            addapt_thresh_subtraction_constant)

In [49]:
svm_test_masks = []
svm_test_masks.extend(pre_processed_test_crack)
svm_test_masks.extend(pre_processed_test_non_crack)
svm_test_masks = list(map(lambda x:x.flatten(), svm_test_masks))

In [51]:
svm_test_labels = []
svm_test_labels.extend(np.ones((len(pre_processed_test_crack), ), np.uint8))
svm_test_labels.extend(np.zeros((len(pre_processed_test_non_crack), ), np.uint8))

In [50]:
svm_test_images_prediction = svm_classifier.predict(svm_test_masks)

In [52]:
accuracy = accuracy_score(svm_test_images_prediction, svm_test_labels)
accuracy

0.6178343949044586

In [78]:
crack_images_for_test_list = []
for image_name in crack_images_for_test_name_list:
    im = cv2.imread('dataset/test/crack/'+ image_name, cv2.IMREAD_GRAYSCALE)
    im_resized = cv2.resize(im, (image_width, image_heigth))
    im_blur = cv2.medianBlur(im_resized, 3)
    crack_images_for_test_list.append(im_blur)
    
noncrack_images_for_test_list = []
for image_name in noncrack_images_for_test_name_list:
    im = cv2.imread('dataset/test/non-crack(vamos_usar)/'+ image_name, cv2.IMREAD_GRAYSCALE)
    im_resized = cv2.resize(im, (image_width, image_heigth))
    im_blur = cv2.medianBlur(im_resized, 3)
    noncrack_images_for_test_list.append(im_blur)

In [79]:
clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(30,30))

test_posprocessed_crack = []
for image in crack_images_for_test_list:
    clahe_image = clahe.apply(image)
    min_pixel_value = np.min(clahe_image)
    darker_image = clahe_image - min_pixel_value
    erosion_image = cv2.erode(darker_image, element, iterations = 3)
    dilated_image = cv2.dilate(erosion_image, element, iterations = 2) # era 2 iteracoes
    image_threshold = cv2.adaptiveThreshold(dilated_image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
    threshold_not = cv2.bitwise_not(image_threshold)
    test_posprocessed_crack.append(threshold_not)

test_posprocessed_non_crack = []
for image in noncrack_images_for_test_list:
    clahe_image = clahe.apply(image)
    min_pixel_value = np.min(clahe_image)
    darker_image = clahe_image - min_pixel_value
    erosion_image = cv2.erode(darker_image, element, iterations = 3)
    dilated_image = cv2.dilate(erosion_image, element, iterations = 2) # era 2 iteracoes
    image_threshold = cv2.adaptiveThreshold(dilated_image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
    threshold_not = cv2.bitwise_not(image_threshold)
    test_posprocessed_non_crack.append(threshold_not)


In [ ]:
crack_images_list = []
for image_name in crack_images_name_list:
    im = cv2.imread('dataset/train/crack/'+ image_name, cv2.IMREAD_GRAYSCALE)
    im_resized = cv2.resize(im, (image_width, image_heigth))
    im_blur = cv2.medianBlur(im_resized, 3)
    crack_images_list.append(im_blur)
    
noncrack_images_list = []
for image_name in noncrack_images_name_list:
    im = cv2.imread('dataset/train/non-crack(vamos_usar)/'+ image_name, cv2.IMREAD_GRAYSCALE)
    im_resized = cv2.resize(im, (image_width, image_heigth))
    im_blur = cv2.medianBlur(im_resized, 3)
    noncrack_images_list.append(im_blur)

In [4]:
image_width = 256
image_heigth = 256

In [5]:
crack_images_list = []
for image_name in crack_images_name_list:
    im = cv2.imread('dataset/train/crack/'+ image_name, cv2.IMREAD_GRAYSCALE)
    im_resized = cv2.resize(im, (image_width, image_heigth))
    im_blur = cv2.medianBlur(im_resized, 3)
    crack_images_list.append(im_blur)
    
noncrack_images_list = []
for image_name in noncrack_images_name_list:
    im = cv2.imread('dataset/train/non-crack(vamos_usar)/'+ image_name, cv2.IMREAD_GRAYSCALE)
    im_resized = cv2.resize(im, (image_width, image_heigth))
    im_blur = cv2.medianBlur(im_resized, 3)
    noncrack_images_list.append(im_blur)

In [7]:
#plt.imshow(crack_images_list[210], cmap='gray')

In [8]:
# usamos
clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(30,30))

clahe_crack = []
for image in crack_images_list:
    clahe_crack.append(clahe.apply(image))

clahe_non_crack = []
for image in noncrack_images_list:
    clahe_non_crack.append(clahe.apply(image))

#save_single_channel_processed_images_for_comparison(crack_images_list, \
#                                                    crack_images_name_list, \
#                                                    clahe_processed, \
#                                                    'clahe')

#image_test_clahe = crack_images_list[210].copy()
#enhanced_img = clahe.apply(image_test_clahe)
#enhanced_img = cv2.cvtColor(image_test_clahe, cv2.COLOR_GRAY2RGB)

#result = np.hstack((image_test_clahe, enhanced_img))
#cv2.imshow('Result', result)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [8]:
# nao usamos
thresholded_clahe_crack_images = []
for image in clahe_processed:
    ret, thresholded = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    thresholded_clahe_crack_images.append(thresholded)

In [11]:
# nao usamos
save_single_channel_processed_images_for_comparison(clahe_processed, \
                                                    crack_images_name_list, \
                                                    thresholded_clahe_crack_images, \
                                                    'clahe-otsu')

In [9]:
# usamos
darker_clahe_crack = []
for image in clahe_crack:
    min_pixel_value = np.min(image)
    darker_image = image - min_pixel_value
    darker_clahe_crack.append(darker_image)
    
darker_clahe_non_crack = []
for image in clahe_non_crack:
    min_pixel_value = np.min(image)
    darker_image = image - min_pixel_value
    darker_clahe_non_crack.append(darker_image)
    
#save_single_channel_processed_images_for_comparison(clahe_processed, \
#                                                    crack_images_name_list, \
#                                                    darker_clahe, \
#                                                    'clahe-darker')

In [10]:
# usamos
erosion_size = 2
element = cv2.getStructuringElement(cv2.MORPH_CROSS, (2 * erosion_size + 1, 2 * erosion_size + 1), (erosion_size, erosion_size))

eroded_darker_clahe_crack = []
for image in darker_clahe_crack:
    erosion_dst = cv2.erode(image, element, iterations = 3)
    eroded_darker_clahe_crack.append(erosion_dst)
    
eroded_darker_clahe_non_crack = []
for image in darker_clahe_non_crack:
    erosion_dst = cv2.erode(image, element, iterations = 3)
    eroded_darker_clahe_non_crack.append(erosion_dst)

In [24]:
#save_single_channel_processed_images_for_comparison(darker_clahe, \
#                                                    crack_images_name_list, \
#                                                    eroded_darker_clahe, \
#                                                    'clahe-darker-eroded')

In [64]:
# usamos
dilated_eroded_darker_clahe_crack = []
for image in eroded_darker_clahe_crack:
    dilated_dst = cv2.dilate(image, element, iterations = 2) # era 2 iteracoes
    dilated_eroded_darker_clahe_crack.append(dilated_dst)

dilated_eroded_darker_clahe_non_crack = []
for image in eroded_darker_clahe_non_crack:
    dilated_dst = cv2.dilate(image, element, iterations = 2) # era 2
    dilated_eroded_darker_clahe_non_crack.append(dilated_dst)

In [38]:
# nao tinha essa etapa aqui
eroded_dilated_eroded_darker_clahe_crack = []
for image in dilated_eroded_darker_clahe_crack:
    erosion_dst = cv2.erode(image, element, iterations = 2)
    eroded_dilated_eroded_darker_clahe_crack.append(erosion_dst)
    
eroded_dilated_eroded_darker_clahe_non_crack = []
for image in dilated_eroded_darker_clahe_non_crack:
    erosion_dst = cv2.erode(image, element, iterations = 2)
    eroded_dilated_eroded_darker_clahe_non_crack.append(erosion_dst)

In [39]:
# teste
kernel = np.ones((3,3),np.uint8)
gradmorfol_eroded_dilated_eroded_darker_clahe_crack = []
for image in eroded_dilated_eroded_darker_clahe_crack:
    gradient = cv2.morphologyEx(image, cv2.MORPH_GRADIENT, kernel)
    gradmorfol_eroded_dilated_eroded_darker_clahe_crack.append(gradient)

gradmorfol_eroded_dilated_eroded_darker_clahe_non_crack = []
for image in eroded_dilated_eroded_darker_clahe_non_crack:
    gradient = cv2.morphologyEx(image, cv2.MORPH_GRADIENT, kernel)
    gradmorfol_eroded_dilated_eroded_darker_clahe_non_crack.append(gradient)

In [40]:
save_single_channel_processed_images_for_comparison(eroded_dilated_eroded_darker_clahe_crack, \
                                                    crack_images_name_list, \
                                                    gradmorfol_eroded_dilated_eroded_darker_clahe_crack, \
                                                    'gradmorfol_eroded_dilated_eroded_darker_clahe_crack')
save_single_channel_processed_images_for_comparison(eroded_dilated_eroded_darker_clahe_non_crack, \
                                                    crack_images_name_list, \
                                                    gradmorfol_eroded_dilated_eroded_darker_clahe_non_crack, \
                                                    'gradmorfol_eroded_dilated_eroded_darker_clahe_non_crack')

In [36]:
# nao usamos
binary_masks_1 = []
for image in dilated_eroded_darker_clahe:
    threshold_value = 60  # Valor de limiar a ser definido
    binary_edges = (image < threshold_value).astype(np.uint8)*255
    binary_masks_1.append(binary_edges)

In [37]:
# nao usamos
save_single_channel_processed_images_for_comparison(dilated_eroded_darker_clahe, \
                                                    crack_images_name_list, \
                                                    binary_masks_1, \
                                                    'clahe-darker-eroded-dilated-binary-60')

In [65]:
# usamos
adaptthresh_dilated_eroded_darker_clahe_crack = []
for image in dilated_eroded_darker_clahe_crack:
    image_threshold = \
        cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
    adaptthresh_dilated_eroded_darker_clahe_crack.append(cv2.bitwise_not(image_threshold))
    
adaptthresh_dilated_eroded_darker_clahe_non_crack = []
for image in dilated_eroded_darker_clahe_non_crack:
    image_threshold = \
        cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
    adaptthresh_dilated_eroded_darker_clahe_non_crack.append(cv2.bitwise_not(image_threshold))

In [29]:
save_single_channel_processed_images_for_comparison(dilated_eroded_darker_clahe_non_crack, \
                                                    noncrack_images_name_list, \
                                                    adaptthresh_dilated_eroded_darker_clahe_non_crack, \
                                                    'adaptthresh_dilated_eroded_darker_clahe_non_crack')

In [30]:
eroded_adaptthresh_dilated_eroded_darker_clahe_crack = []
for image in adaptthresh_dilated_eroded_darker_clahe_crack:
    erosion_dst = cv2.erode(image, element, iterations = 1)
    eroded_adaptthresh_dilated_eroded_darker_clahe_crack.append(erosion_dst)
    
eroded_adaptthresh_dilated_eroded_darker_clahe_non_crack = []
for image in adaptthresh_dilated_eroded_darker_clahe_non_crack:
    erosion_dst = cv2.erode(image, element, iterations = 1)
    eroded_adaptthresh_dilated_eroded_darker_clahe_non_crack.append(erosion_dst)

In [31]:
# usamos
conncomp_eroded_adaptthresh_dilated_eroded_darker_clahe_crack = []
for image in eroded_adaptthresh_dilated_eroded_darker_clahe_crack:
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(image)
    
    areas = [stats[i, cv2.CC_STAT_AREA] for i in range(1, num_labels)]
    max_area = np.max(areas) * 0.2
    for i in range(1, num_labels):
        if stats[i, cv2.CC_STAT_AREA] < max_area:
            labels[labels == i] = 0
    
    posproc_image = (labels > 0).astype(np.uint8) * 255
    conncomp_eroded_adaptthresh_dilated_eroded_darker_clahe_crack.append(posproc_image)
    
conncomp_eroded_adaptthresh_dilated_eroded_darker_clahe_non_crack = []
for image in eroded_adaptthresh_dilated_eroded_darker_clahe_non_crack:
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(image)
    
    areas = [stats[i, cv2.CC_STAT_AREA] for i in range(1, num_labels)]
    max_area = np.max(areas) * 0.2
    for i in range(1, num_labels):
        if stats[i, cv2.CC_STAT_AREA] < max_area:
            labels[labels == i] = 0
    
    posproc_image = (labels > 0).astype(np.uint8) * 255
    conncomp_eroded_adaptthresh_dilated_eroded_darker_clahe_non_crack.append(posproc_image)

In [32]:
save_images(conncomp_eroded_adaptthresh_dilated_eroded_darker_clahe_crack, crack_images_name_list, 'preprocessed_crack_1')
save_images(conncomp_eroded_adaptthresh_dilated_eroded_darker_clahe_non_crack, noncrack_images_name_list, 'preprocessed_non_crack_1')

In [46]:
# IA

In [80]:
svm_input_masks_test = []
svm_input_masks_test.extend(test_posprocessed_crack)
svm_input_masks_test.extend(test_posprocessed_non_crack)
svm_input_labels_test = []
svm_input_labels_test.extend(np.ones((len(test_posprocessed_crack),), np.uint8))
svm_input_labels_test.extend(np.zeros((len(test_posprocessed_non_crack),), np.uint8))

svm_input_masks_test = list(map(lambda x:x.flatten(), svm_input_masks_test))

In [103]:
import joblib

In [104]:
model_filename = 'svm_model.pkl'
joblib.dump(svm_classifier, model_filename)

['svm_model.pkl']

In [85]:
from sklearn.ensemble import RandomForestClassifier

In [86]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(svm_input_masks, svm_input_labels)

RandomForestClassifier(random_state=42)

In [87]:
predictions = rf_classifier.predict(svm_input_masks_test)

In [88]:
accuracy2 = accuracy_score(svm_input_labels_test, predictions)
accuracy2

0.5987261146496815

In [89]:
from sklearn.neighbors import KNeighborsClassifier

In [100]:
knn_classifier = KNeighborsClassifier(n_neighbors=2)
knn_classifier.fit(svm_input_masks, svm_input_labels)

KNeighborsClassifier(n_neighbors=2)

In [101]:
predictions2 = knn_classifier.predict(svm_input_masks_test)

C:\Users\ester\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [102]:
accuracy3 = accuracy_score(svm_input_labels_test, predictions2)
accuracy3

0.5095541401273885

In [38]:
# SOBEL OPERATOR
sobel_dilated_eroded_darker_clahe = []
for image in dilated_eroded_darker_clahe:
    gradient_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    gradient_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    gradient_magnitude = np.sqrt(gradient_x**2 + gradient_y**2)
    gradient_magnitude = cv2.convertScaleAbs(gradient_magnitude)
    
    sobel_dilated_eroded_darker_clahe.append(gradient_magnitude)

In [39]:
save_single_channel_processed_images_for_comparison(dilated_eroded_darker_clahe, \
                                                    crack_images_name_list, \
                                                    sobel_dilated_eroded_darker_clahe, \
                                                    'clahe-darker-eroded-dilated-sobel')

In [40]:
binary_masks_2 = []
for image in sobel_dilated_eroded_darker_clahe:
    threshold_value = 100  # Valor de limiar a ser definido
    binary_edges = (image < threshold_value).astype(np.uint8)*255
    binary_masks_2.append(binary_edges)

In [41]:
save_single_channel_processed_images_for_comparison(sobel_dilated_eroded_darker_clahe, \
                                                    crack_images_name_list, \
                                                    binary_masks_2, \
                                                    'clahe-darker-eroded-dilated-sobel-binary-100')

In [43]:
laplacian_dilated_eroded_darker_clahe = []
for image in dilated_eroded_darker_clahe:
    laplacian = cv2.Laplacian(image, cv2.CV_64F)
    laplacian = np.uint8(np.absolute(laplacian))
    laplacian_dilated_eroded_darker_clahe.append(laplacian)

In [44]:
save_single_channel_processed_images_for_comparison(dilated_eroded_darker_clahe, \
                                                    crack_images_name_list, \
                                                    laplacian_dilated_eroded_darker_clahe, \
                                                    'clahe-darker-eroded-dilated-laplacian')

In [72]:
save_single_channel_processed_images_for_comparison(dilated_eroded_darker_clahe, \
                                                    crack_images_name_list, \
                                                    adaptthresh_dilated_eroded_darker_clahe, \
                                                    'clahe-darker-eroded-dilated-adaptthresh-11-2')

In [101]:
eroded_adaptthresh_dilated_eroded_darker_clahe = []
erosion_size = 2
element = cv2.getStructuringElement(cv2.MORPH_CROSS, (2 * erosion_size + 1, 2 * erosion_size + 1), (erosion_size, erosion_size))
for image in adaptthresh_dilated_eroded_darker_clahe:
    erosion_dst = cv2.erode(image, element, iterations = 1)
    eroded_adaptthresh_dilated_eroded_darker_clahe.append(erosion_dst)

In [78]:
medianfilter_adaptthresh_dilated_eroded_darker_clahe = []
for image in adaptthresh_dilated_eroded_darker_clahe:
    median = cv2.medianBlur(image, 5)
    medianfilter_adaptthresh_dilated_eroded_darker_clahe.append(median)

In [74]:
save_single_channel_processed_images_for_comparison(adaptthresh_dilated_eroded_darker_clahe, \
                                                    crack_images_name_list, \
                                                    medianfilter_adaptthresh_dilated_eroded_darker_clahe, \
                                                    'clahe-darker-eroded-dilated-adaptthresh-11-2-medianfilter')

In [109]:
save_single_channel_processed_images_for_comparison(eroded_adaptthresh_dilated_eroded_darker_clahe, \
                                                    crack_images_name_list, \
                                                    conncomp_adaptthresh_dilated_eroded_darker_clahe, \
                                                    'clahe-darker-eroded-dilated-adaptthresh-11-2-conncomp-2')

In [50]:
enclosure_adaptthresh_dilated_eroded_darker_clahe = []
kernel = np.ones((4, 4), np.uint8)
for image in adaptthresh_dilated_eroded_darker_clahe:
    imagem_sem_ruido = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    enclosure_adaptthresh_dilated_eroded_darker_clahe.append(imagem_sem_ruido)

In [51]:
save_single_channel_processed_images_for_comparison(adaptthresh_dilated_eroded_darker_clahe, \
                                                    crack_images_name_list, \
                                                    enclosure_adaptthresh_dilated_eroded_darker_clahe, \
                                                    'clahe-darker-eroded-dilated-adaptthresh-11-2-enclosure')

In [252]:
save_single_channel_processed_images_for_comparison(sobel_eroded_clahe, \
                                                    crack_images_name_list, \
                                                    binary_masks_1, \
                                                    'binary-sobel-eroded-clahe')

In [212]:
canny_clahe = []
for image_clahe in clahe_processed:
    threshold_value, _ = cv2.threshold(image_clahe, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    canny_res = cv2.Canny(image_clahe, np.round(threshold_value * 0.5), np.round(threshold_value))
    canny_clahe.append(canny_res)

In [213]:
save_single_channel_processed_images_for_comparison(crack_images_list, \
                                                    crack_images_name_list, \
                                                    canny_clahe, \
                                                    'canny-clahe')

In [181]:
cv2.imshow('image', crack_images_list[0])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [120]:
new_img = np.zeros((256, 256, 3)).astype(np.uint8)
new_img[:,:,0] = crack_images_list[3][:,:,0]
new_img[:,:,1] = (crack_images_list[3][:,:,1]*1.5).astype(np.uint8)
new_img[:,:,2] = crack_images_list[3][:,:,2] #cv2.equalizeHist(

cv2.imshow('contrasted', cv2.cvtColor(new_img, cv2.COLOR_HSV2RGB))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
images_hsi = []
for image in crack_images_list:
    image_matrix = np.array(image)
    h_matrix, s_matrix, i_matrix = converter_RGB_para_HSI(image_matrix)
    images_hsi.append([h_matrix, s_matrix, i_matrix])

In [31]:
#fig, axes = plt.subplots(1, 3, figsize=(10, 5))
#cv2.imshow('Hue', (images_hsi[3][0] / (2 * np.pi)).astype(np.uint8))
#cv2.imshow('Saturation', (images_hsi[3][1] * 255).astype(np.uint8))
#cv2.imshow('Intensity', (images_hsi[3][2] * 255).astype(np.uint8))
#cv2.waitKey(0)
#cv2.destroyAllWindows()

hsi_image = np.zeros((256, 256, 3)).astype(np.uint8)
hsi_image[:,:,0] = ((images_hsi[1][0] / (2 * np.pi)) * 255).astype(np.uint8)
hsi_image[:,:,1] = (images_hsi[1][1] * 255).astype(np.uint8)
hsi_image[:,:,2] = (images_hsi[1][2] * 255).astype(np.uint8)

hue = hsi_image[:, :, 0]
saturation = hsi_image[:, :, 1]
intensity = hsi_image[:, :, 2]

contrast_factor = 12  # Adjust this value to control the contrast enhancement
enhanced_intensity = np.clip(intensity * contrast_factor, 0, 255).astype(np.uint8)

# Create the enhanced HSI image by combining the adjusted intensity with the original hue and saturation
enhanced_hsi_image = np.stack((hue, saturation, enhanced_intensity), axis=-1)

#enhanced_hsi_image

cv2.imshow('hsi', enhanced_hsi_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

#axes[0].imshow((images_hsi[1][0] / (2 * np.pi)).astype(np.uint8), cmap='gray')
#axes[1].imshow((images_hsi[1][1] * 255).astype(np.uint8), cmap='gray')
#axes[2].imshow((images_hsi[1][2] * 255).astype(np.uint8), cmap='gray')

In [56]:
print(crack_images_list[0].shape)
new_img = np.zeros((256, 256, 3)).astype(np.uint8)
for x in range(image_width):
    for y in range(image_heigth):
        new_img[x,y,:] = colorsys.rgb_to_hsv(crack_images_list[0][x][y][0],crack_images_list[0][x][y][1],crack_images_list[0][x][y][2])

(256, 256, 3)


In [54]:
new_img.shape

(256, 256, 3)

In [57]:
cv2.imshow('hsi', new_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [24]:
# OTSU THRESHOLD
thresholded_crack_images = []
for image in crack_images_list:
    ret, thresholded = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    thresholded_crack_images.append(thresholded)

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\thresh.cpp:1555: error: (-2:Unspecified error) in function 'double __cdecl cv::threshold(const class cv::_InputArray &,const class cv::_OutputArray &,double,double,int)'
> THRESH_OTSU mode:
>     'src_type == CV_8UC1 || src_type == CV_16UC1'
> where
>     'src_type' is 16 (CV_8UC3)


In [28]:
def save_single_channel_processed_images_for_comparison(original_images_list, original_images_names_list, \
                                                        processed_images_with_single_channel, \
                                                        directory_to_save):
    height = original_images_list[0].shape[0]
    width = original_images_list[0].shape[1]
    
    for idx in range(len(original_images_names_list)):
        
        collage_width = original_images_list[idx].shape[1] * 2
        collage = np.ones((height, collage_width), np.uint8) * 255
        collage = cv2.resize(collage, (collage_width, height))
        
        collage[:, :original_images_list[idx].shape[1]] = original_images_list[idx]
        collage[:, original_images_list[idx].shape[1]:] = processed_images_with_single_channel[idx]
        
        if not os.path.exists('processed-dataset-tests/' + directory_to_save):
            os.makedirs('processed-dataset-tests/' + directory_to_save)
        cv2.imwrite('processed-dataset-tests/' + directory_to_save + '/' + original_images_names_list[idx], collage)

In [78]:
save_single_channel_processed_images_for_comparison(crack_images_list, \
                                                    crack_images_name_list, \
                                                    thresholded_crack_images, \
                                                    'only-otsu')

In [82]:
# SOBEL OPERATOR
sobel_images = []
canny_images = []
for image in crack_images_list:
    gradient_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    gradient_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    gradient_magnitude = np.sqrt(gradient_x**2 + gradient_y**2)
    gradient_magnitude = cv2.convertScaleAbs(gradient_magnitude)
    
    sobel_images.append(gradient_magnitude)
    
    canny_edges = cv2.Canny(gradient_magnitude, threshold1=50, threshold2=150)
    
    canny_images.append(canny_edges)

In [83]:
save_single_channel_processed_images_for_comparison(crack_images_list, \
                                                    crack_images_name_list, \
                                                    sobel_images, \
                                                    'only-sobel')

In [85]:
save_single_channel_processed_images_for_comparison(crack_images_list, \
                                                    crack_images_name_list, \
                                                    canny_images, \
                                                    'only-canny')

In [87]:
# SOBEL + OTSU THRESHOLD
sobel_otsu_crack_images = []
for image in sobel_images:
    ret, thresholded = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    sobel_otsu_crack_images.append(thresholded)

In [88]:
save_single_channel_processed_images_for_comparison(crack_images_list, \
                                                    crack_images_name_list, \
                                                    sobel_otsu_crack_images, \
                                                    'sobel-and-otsu')